In [1]:
import os
import json
import shutil
import random
import cv2
import tqdm
PATH_IN_1 = "./data/imagenet/ILSVRC/Data/CLS-LOC/train/"
PATH_IN_2 = "./data/dataset/mini-imagenet/training/red_noise_nl_0.0"
PATH_OUT = "./data/dataset/LT_with_noise/selected_imagenet/"
PATH_IMAGENET_NOISE = "./data/dataset/LT_with_noise/imagenet_noise/"
PATH_NOISE = "./data/dataset/mini-imagenet/all_images"

In [3]:

# change the form of split  0.1

noise_dict = {}

with open ('./data/dataset/mini-imagenet/split/red_noise_nl_0.8.txt') as f:
    word_list = f.readlines()

for word in word_list:
    temp_list = word.split()
    if temp_list[1] not in noise_dict:
        noise_dict[temp_list[1]] = []
    if temp_list[0][0] != 'n':
        noise_dict[temp_list[1]].append(temp_list[0])


In [2]:
# construct clean dataset and resize 
# 1000 pic per cls
for cls in os.listdir(PATH_IN_2):
    os.mkdir(os.path.join(PATH_OUT,cls))
    temp_path = os.path.join(PATH_IN_2,cls)
    # get cls id
    selected_sample = os.listdir(temp_path)[0]
    cls_id = selected_sample[:9]
    choose_path = os.path.join(PATH_IN_1,cls_id)
    name_list = os.listdir(choose_path)
    random.shuffle(name_list)
    selected_list = name_list[:1000]
    for name in selected_list:
        path_in = os.path.join(choose_path,name)
        path_out = os.path.join(PATH_OUT,cls)
        shutil.copy(path_in,path_out)






In [3]:
# rename and resize
IMG_SIZE = 84
for cls in os.listdir(PATH_OUT):
    temp_path = os.path.join(PATH_OUT,cls)
    for file_name in os.listdir(temp_path):
        portion = os.path.splitext(file_name)
        new_name = portion[0]+".jpg"
        if portion[1] == ".JPEG" :
            os.rename(os.path.join(PATH_OUT,cls,file_name),os.path.join(PATH_OUT,cls,new_name))
        img_array=cv2.imread(os.path.join(PATH_OUT,cls,new_name),cv2.IMREAD_COLOR)
        new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        save_path = os.path.join(PATH_OUT,cls,new_name)
        cv2.imwrite(save_path,new_array)



In [7]:
# construct with-noise-dataset without long-tail
# An example of 0.3 noise_ratio
pic_len = 1000
noise_rate_list = [0.3]
for noise_rate in noise_rate_list:
    sample_len = int(pic_len *(1-noise_rate))
    noise_len = int(pic_len*noise_rate)
    for key,value in noise_dict.items():
        key = str(key)
        if len(key) ==1:
            key = '0'+key
        os.mkdir(os.path.join(PATH_IMAGENET_NOISE,str(noise_rate),key))
        random.shuffle(value)
        # add noise
        if len(value) > noise_len:
            selected_noise_list = value[:noise_len]
        else:
            selected_noise_list = value[:len(value)]
        
        for selected_noise in selected_noise_list:
            path_in = os.path.join(PATH_NOISE,selected_noise)
            path_out = os.path.join(PATH_IMAGENET_NOISE,str(noise_rate),key)
            shutil.copy(path_in,path_out)
        # add sample
        temp_path = os.path.join(PATH_OUT,key)
        sample_list = os.listdir(temp_path)
        random.shuffle(sample_list)
        selected_sample_list = sample_list[:sample_len]
        for selected_sample in selected_sample_list:
            path_in = os.path.join(PATH_OUT,key,selected_sample)
            path_out = os.path.join(PATH_IMAGENET_NOISE,str(noise_rate),key)
            shutil.copy(path_in,path_out)

